In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

In [5]:
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm

class KAN_PINN_Pipeline:
    def __init__(self, kan_path, pinn_model_class, pinn_config, device="cuda"):
        """
        A modular pipeline integrating KAN and PINN.

        Args:
            kan_path: Path to the pre-trained KAN (.pth) model.
            pinn_model_class: The class of the PINN model (e.g., BurgersPINN, HeatPINN).
            pinn_config: Dictionary containing PINN-specific parameters.
            device: "cuda" or "cpu".
        """
        self.device = torch.device(device)
        
        # Load KAN model
        self.kan = torch.load(kan_path, map_location=self.device)
        self.kan.eval()  # Ensure KAN is in evaluation mode

        # Load PINN model
        self.pinn = pinn_model_class(**pinn_config).to(self.device)
        self.optimizer = torch.optim.Adam(self.pinn.parameters(), lr=1e-3)
        self.criterion = nn.MSELoss()

    def apply_kan(self, noisy_x):
        """
        Process noisy data through the pre-trained KAN.

        Args:
            noisy_x: Noisy input data.

        Returns:
            Cleaned data from KAN.
        """
        x_tensor = torch.tensor(noisy_x, dtype=torch.float32).to(self.device)
        with torch.no_grad():
            clean_x = self.kan(x_tensor)
        return clean_x.cpu().numpy()

    def train_pinn(self, x, t, u, train_fn, use_kan=True, epochs=1000):
        """
        Train the PINN with or without KAN-processed input.

        Args:
            x: Spatial coordinates.
            t: Time values.
            u: Ground truth.
            train_fn: Training function for the PINN.
            use_kan: If True, applies KAN before training.
            epochs: Number of training iterations.
        """
        # Apply KAN if needed
        if use_kan:
            x = self.apply_kan(x)
        
        x_tensor = torch.tensor(x, dtype=torch.float32).to(self.device)
        t_tensor = torch.tensor(t, dtype=torch.float32).to(self.device)
        u_tensor = torch.tensor(u, dtype=torch.float32).to(self.device)

        train_fn(self.pinn, x_tensor, t_tensor, u_tensor, self.optimizer, self.criterion, epochs)

    def predict_pinn(self, x, t, use_kan=True):
        """
        Make predictions using the trained PINN.

        Args:
            x: Spatial coordinates.
            t: Time values.
            use_kan: If True, applies KAN before prediction.

        Returns:
            PINN output as numpy array.
        """
        if use_kan:
            x = self.apply_kan(x)
        
        x_tensor = torch.tensor(x, dtype=torch.float32).to(self.device)
        t_tensor = torch.tensor(t, dtype=torch.float32).to(self.device)

        with torch.no_grad():
            prediction = self.pinn(torch.cat((x_tensor, t_tensor), dim=1))

        return prediction.cpu().numpy()

    def compute_metrics(self, x, t, u_clean, u_noisy):
        """
        Compute required metrics for evaluation.

        Args:
            x: Spatial coordinates.
            t: Time values.
            u_clean: Ground truth solution.
            u_noisy: Noisy dataset.

        Returns:
            Dictionary of computed metrics.
        """
        # Without KAN
        pinn_pred_no_kan = self.predict_pinn(x, t, use_kan=False)
        pinn_loss_no_kan = self.criterion(torch.tensor(pinn_pred_no_kan), torch.tensor(u_clean)).item()

        # With KAN
        pinn_pred_with_kan = self.predict_pinn(x, t, use_kan=True)
        pinn_loss_with_kan = self.criterion(torch.tensor(pinn_pred_with_kan), torch.tensor(u_clean)).item()

        # MSE Metrics
        mse_clean_predicted = np.mean((u_clean - pinn_pred_with_kan) ** 2)
        mse_clean_noisy = np.mean((u_clean - u_noisy) ** 2)

        return {
            "pinn_loss_no_kan": pinn_loss_no_kan,
            "pinn_loss_with_kan": pinn_loss_with_kan,
            "mse_clean_vs_predicted": mse_clean_predicted,
            "mse_clean_vs_noisy": mse_clean_noisy,
        }

In [7]:
class BurgersPINN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PINN, self).__init__()
        self.layers = nn.ModuleList(
            [
                (
                    nn.Linear(input_size if i == 0 else hidden_size, hidden_size)
                    if i % 2 == 0
                    else nn.Tanh()
                )
                for i in range(20)
            ]
        )
        self.layers.append(nn.Linear(hidden_size, output_size))
        self.loss = nn.MSELoss()
        self.lambda2 = nn.Parameter(
            torch.tensor([0.01], dtype=torch.float32, device="cuda")
        )
        self.lambda1 = nn.Parameter(
            torch.tensor([1.0], dtype=torch.float32, device="cuda")
        )
        self.optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        self.optimizer.param_groups[0]["params"].append(self.lambda1)
        self.optimizer.param_groups[0]["params"].append(self.lambda2)

    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def loss_fn(self, x, u):
        u_pred = self.forward(x)
        return self.loss(u_pred, u)

    def residual_loss(self, xtrain, fhat):
        x = xtrain[:, 0]
        t = xtrain[:, 1]
        g = xtrain.clone()
        g.requires_grad = True
        u_pred = self.forward(g)
        u_x_t = torch.autograd.grad(
            u_pred,
            g,
            torch.ones([xtrain.shape[0], 1]).to("cuda"),
            retain_graph=True,
            create_graph=True,
        )[0]
        u_xx_tt = torch.autograd.grad(
            u_x_t, g, torch.ones(xtrain.shape).to("cuda"), create_graph=True
        )[0]
        u_x = u_x_t[:, [0]]
        u_t = u_x_t[:, [1]]
        u_xx = u_xx_tt[:, [0]]
        return self.loss(u_t + (self.lambda1 * u_pred * u_x) - (self.lambda2 * u_xx), fhat)

    def total_loss(self, xtrain, utrain, fhat):
        return self.loss_fn(xtrain, utrain) + self.residual_loss(xtrain, fhat)

    def train_model(self, xtrain, utrain, epochs=1000):
        fhat = torch.zeros(xtrain.shape[0], 1, device="cuda")
        for epoch in tqdm(range(epochs)):
            self.optimizer.zero_grad()
            loss = self.total_loss(xtrain, utrain, fhat)
            loss.backward()
            self.optimizer.step()
            if epoch % 1000 == 0:
                print(f"Epoch {epoch}, Loss {loss.item()}, Lambda2 (Nu) {self.lambda2.item()}, Lambda1 {self.lambda1.item()}")

In [ ]:
pinn_config = {"input_size": 2, "hidden_size": 20, "output_size": 1}
pipeline = KAN_PINN_Pipeline("Thisshitworks.pth", BurgersPINN, pinn_config)

# Load dataset
x_noisy = np.load("x_noisy.npy")
t = np.load("t.npy")
u_clean = np.load("burgerClean.npy")
u_noisy = np.load("burgerNoisy.npy")

# Train without KAN
pipeline.train_pinn(x_noisy, t, u_clean, train_burgers_pinn, use_kan=False, epochs=5000)

# Train with KAN
pipeline.train_pinn(x_noisy, t, u_clean, train_burgers_pinn, use_kan=True, epochs=5000)

# Compute metrics
metrics = pipeline.compute_metrics(x_noisy, t, u_clean, u_noisy)
print(metrics)